# <center>Car Maker Classification (MobileNet Keras)</center>

Google drive mount

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


dowonloading dataset

In [28]:
cd /content/drive/MyDrive/Colab\ Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [30]:
!wget http://www.awlaa.app/cars-dataset.zip

--2021-11-23 18:50:56--  http://www.awlaa.app/cars-dataset.zip
Resolving www.awlaa.app (www.awlaa.app)... 51.144.85.121
Connecting to www.awlaa.app (www.awlaa.app)|51.144.85.121|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.awlaa.app/cars-dataset.zip [following]
--2021-11-23 18:50:56--  https://www.awlaa.app/cars-dataset.zip
Connecting to www.awlaa.app (www.awlaa.app)|51.144.85.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1959263989 (1.8G) [application/zip]
Saving to: ‘cars-dataset.zip’

cars-dataset.zip    100%[===================>]   1.82G  10.2MB/s    in 3m 14s  

2021-11-23 18:54:11 (9.64 MB/s) - ‘cars-dataset.zip’ saved [1959263989/1959263989]



In [32]:
!unzip cars-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train_spl/74/005938.jpg  
  inflating: dataset/train_spl/74/005939.jpg  
  inflating: dataset/train_spl/74/005942.jpg  
  inflating: dataset/train_spl/74/005946.jpg  
  inflating: dataset/train_spl/74/005947.jpg  
  inflating: dataset/train_spl/74/005948.jpg  
  inflating: dataset/train_spl/74/005950.jpg  
  inflating: dataset/train_spl/74/005953.jpg  
  inflating: dataset/train_spl/74/005957.jpg  
  inflating: dataset/train_spl/74/005958.jpg  
  inflating: dataset/train_spl/74/005961.jpg  
  inflating: dataset/train_spl/74/005964.jpg  
  inflating: dataset/train_spl/74/005966.jpg  
  inflating: dataset/train_spl/74/005969.jpg  
  inflating: dataset/train_spl/74/005970.jpg  
  inflating: dataset/train_spl/74/005973.jpg  
  inflating: dataset/train_spl/74/005974.jpg  
  inflating: dataset/train_spl/74/005975.jpg  
  inflating: dataset/train_spl/74/005977.jpg  
  inflating: dataset/train_spl/74/005980.jpg  
  inflati

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

In [34]:
keras.__version__

'2.7.0'

## Run on GPU

In [35]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


## Preparing images

In [36]:
colab_path = '/content/drive/MyDrive/Colab Notebooks/'

In [37]:
train_path = colab_path + 'dataset/train_spl'
valid_path = colab_path +'dataset/val_spl'
test_path = colab_path + 'dataset/test_spl'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 9791 images belonging to 196 classes.
Found 3312 images belonging to 196 classes.
Found 3082 images belonging to 196 classes.


## Choosing Model

In [38]:
model_original = tf.keras.applications.mobilenet.MobileNet()

In [40]:
# model_original.summary() # to show all models layers

In [41]:
# taking all layers except last 6 layers
x = model_original.layers[-2].output
output = Dense(units=196, activation='softmax')(x)

In [42]:
model = Model(inputs=model_original.input, outputs=output)

In [43]:
# freezing last n layers (n is an arbitrary number)
for layer in model.layers[:-23]:
    layer.trainable = False

In [44]:
# model.summary() # to show all models layers

## Training Model

In [45]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
# makeing checkpoint each epoch

# checkpoint = ModelCheckpoint("M_MobileNet_checkpoints/M_MobileNet_checkpoint-{epoch:02d}.hdf5", monitor='loss', verbose=0,
#     save_best_only=False, mode='auto', save_freq='epoch')

csv_logger = CSVLogger("/content/drive/MyDrive/models_logs/model_mobilenet_log.csv", append=True)

In [47]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=1,
            # callbacks=[checkpoint]
            callbacks=[csv_logger]
)

Epoch 1/30
980/980 [==============================] - 127s 128ms/step - loss: 4.5437 - accuracy: 0.1629 - val_loss: 2.6015 - val_accuracy: 0.3638
Epoch 2/30
980/980 [==============================] - 125s 128ms/step - loss: 1.4359 - accuracy: 0.6307 - val_loss: 1.9656 - val_accuracy: 0.5109
Epoch 3/30
980/980 [==============================] - 126s 128ms/step - loss: 0.6152 - accuracy: 0.8384 - val_loss: 1.6409 - val_accuracy: 0.5839
Epoch 4/30
980/980 [==============================] - 126s 128ms/step - loss: 0.2823 - accuracy: 0.9347 - val_loss: 2.0125 - val_accuracy: 0.5175
Epoch 5/30
980/980 [==============================] - 125s 128ms/step - loss: 0.1735 - accuracy: 0.9626 - val_loss: 1.6177 - val_accuracy: 0.6033
Epoch 6/30
980/980 [==============================] - 125s 128ms/step - loss: 0.1391 - accuracy: 0.9724 - val_loss: 1.5816 - val_accuracy: 0.6350
Epoch 7/30
980/980 [==============================] - 125s 128ms/step - loss: 0.1370 - accuracy: 0.9669 - val_loss: 1.7282 -

## Saving Model

In [48]:
model.save("M_MobileNet.h5")

## Result Visualization

In [ ]:
test_labels = test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [ ]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.rcParams["figure.figsize"] = (12,12)
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")

In [ ]:
# cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [ ]:
# test_batches.class_indices

In [ ]:
# cm_plot_labels = list(range(0, 196))
# plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')